In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import pandas as pd
import os
import subprocess

In [17]:
# Caminho para o chromedriver, se não estiver no PATH
caminho_driver = r"C:\WebDriver\bin\chromedriver.exe"  # Altere se necessário

# Configuração do navegador
options = Options()
#options.add_argument("--start-maximized")
options.add_argument("--headless=new")  # Versão nova do modo headless
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Inicia o navegador
service = Service(caminho_driver)
driver = webdriver.Chrome(service=service, options=options)

# Acessa o Google
driver.get("https://viavarejo.service-now.com/nav_to.do?uri=%2Fsc_task_list.do%3Fsysparm_query%3Dopened_atONThis%20year@javascript:gs.beginningOfThisYear()@javascript:gs.endOfThisYear()%26sysparm_first_row%3D1%26sysparm_view%3Dcsc%26sysparm_fixed_query%3Dassignment_groupDYNAMICd6435e965f510100a9ad2572f2b47744%5EEQ")

In [18]:
time.sleep(3)

# Preenche o e-mail ou usuário
driver.find_element(By.ID, "i0116").send_keys("ricardo.carvalho@viavarejo.com.br")

# Clica no botão "Avançar"
driver.find_element(By.ID, "idSIButton9").click()

In [19]:
time.sleep(3)

# Preenche a senha fixa
senha = "@Rcs'0101"
driver.find_element(By.ID, "i0118").send_keys(senha)
driver.find_element(By.ID, "idSIButton9").click()

In [20]:
time.sleep(3)
driver.find_element(By.ID, "KmsiCheckboxField").click()
driver.find_element(By.ID, "idSIButton9").click()

In [21]:
time.sleep(2)
# 1. Trocar para o iframe que contém a tabela
iframe = driver.find_element(By.TAG_NAME, "iframe")
driver.switch_to.frame(iframe)
print("✅ Switched para o iframe.")

# 2. Achar o cabeçalho "Pai"
elemento_pai = driver.find_element(By.XPATH, "//a[contains(text(), 'Pai')]")

# 3. Clicar com o botão direito
actions = ActionChains(driver)
actions.context_click(elemento_pai).perform()

print("🖱️ Clique com botão direito no cabeçalho 'Pai' realizado.")

✅ Switched para o iframe.
🖱️ Clique com botão direito no cabeçalho 'Pai' realizado.


In [22]:
# Clicar no botão "Exportar"
try:
    exportar_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@data-context-menu-label='Exportar']"))
    )
    exportar_btn.click()
    print("✅ Clique no 'Exportar' realizado com sucesso.")
except Exception as e:
    print(f"⚠️ Erro ao clicar no botão Exportar: {e}")

✅ Clique no 'Exportar' realizado com sucesso.


In [23]:
# Clica na opção "CSV"
try:
    csv_opcao = driver.find_element(By.XPATH, "//div[text()='CSV']")
    driver.execute_script("arguments[0].click();", csv_opcao)
    print("✅ Clique em 'CSV' realizado com sucesso.")
except Exception as e:
    print(f"⚠️ Erro ao clicar em 'CSV': {e}")

✅ Clique em 'CSV' realizado com sucesso.


In [24]:
# Espera até o botão 'Baixar' ficar visível e clicável
try:
    btn_baixar = WebDriverWait(driver, 50).until(
        EC.element_to_be_clickable((By.ID, "download_button"))
    )
    driver.execute_script("arguments[0].click();", btn_baixar)
    print("✅ Clique em 'Baixar' realizado com sucesso.")
except Exception as e:
    print(f"⚠️ Erro ao clicar no botão Baixar: {e}")

✅ Clique em 'Baixar' realizado com sucesso.


In [25]:
#Aguarda 2 segundos para o download do CSV
time.sleep(10)

# Tratativa do csv

# Lê o arquivo CSV
caminho_csv = r"C:\Users\2160038357\Downloads\sc_task.csv"
df = pd.read_csv(caminho_csv, encoding='latin1', dtype=str)

# Remove quebras de linha
df = df.applymap(lambda x: x.replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)

# Renomeia as colunas
df.rename(columns={
        "parent" : "numero_chamado",
        "number" : "numero_scta",
        "opened_at" : "data_hora_abertura",
        "assignment_group" : "grupo_designado",
        "parent.due_date" : "data_prazo",
        "u_resolved_at" : "data_solucao",
        "u_resolved_by" : "resolvido_por_base",
        "u_resolved_by.user_name" : "cd_fun_setor",
        "request_item.cat_item" : "setor_destinado",
        "opened_by" : "aberto_por",
        "u_requested_for" : "solicitado_para",
        "u_requested_for.user_name" : "cd_fun_solicitado",
        "state" : "situacao",
        "close_notes" : "ds_chamado",
        "closed_at" : "data_encerramento",
        "business_service" : "tipo_servico",
        "service_offering" : "oferta_servico",
        "closed_by" : "encerrado_por"        
        }, inplace=True)

C:\Users\2160038357\AppData\Local\Temp\ipykernel_24548\1795842702.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)


In [26]:
# Fecha o navegador
driver.quit()

In [27]:
# Salva CSV limpo (sobrescrevendo)
df.to_csv(caminho_csv, index=False, encoding='latin1', quoting=1)
print("✅ Arquivo limpo e salvo novamente com sucesso.")

✅ Arquivo limpo e salvo novamente com sucesso.


In [28]:
# Envia o arquivo para o DBFS do Databricks

# Caminho da pasta local onde está o arquivo CSV


# Caminho de destino no DBFS
pasta_dbfs = "dbfs:/FileStore/Controladoria_RicardoCarvalho"
arquivo_dbfs = f"{pasta_dbfs}/sc_task.csv"

# Cria a pasta no DBFS (caso ainda não exista)
comando_mkdir = [
    "databricks", "fs", "mkdirs", pasta_dbfs, "--profile", "meu_databricks"
]
subprocess.run(comando_mkdir, capture_output=True, text=True)

# Faz upload do arquivo CSV
comando_upload = [
    "databricks", "fs", "cp", caminho_csv, arquivo_dbfs,
    "--overwrite", "--profile", "meu_databricks"
]
resultado = subprocess.run(comando_upload, capture_output=True)

# Verifica o resultado do upload
if resultado.returncode == 0:
    print(f"✅ Arquivo CSV enviado com sucesso para o DBFS: {arquivo_dbfs}")
else:
    print("❌ Erro ao enviar arquivo CSV para o DBFS:")
    print(resultado.stderr.decode('utf-8', errors='replace'))

✅ Arquivo CSV enviado com sucesso para o DBFS: dbfs:/FileStore/Controladoria_RicardoCarvalho/sc_task.csv


In [29]:
#Exclui arquivo nos donwloads

# Verifica se o arquivo existe e exclui
if os.path.exists(caminho_csv):
    os.remove(caminho_csv)
    print(f"✅ Arquivo excluído: {caminho_csv}")
else:
    print(f"⚠️ Arquivo não encontrado: {caminho_csv}")

✅ Arquivo excluído: C:\Users\2160038357\Downloads\sc_task.csv
